<a href="https://colab.research.google.com/github/hwan0309/DataAnalyst_T/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ls

sample_data/


In [25]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"


import warnings
warnings.filterwarnings(action="ignore")

import pandas as pd
import numpy as np
import random
import tensorflow as tf

from tqdm import tqdm
from IPython.display import clear_output
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

from transformers import AutoTokenizer, TFAutoModel, TFDistilBertModel

ModuleNotFoundError: No module named 'silence_tensorflow'

In [3]:
SEED = 42
MODEL_NAME = "HWAN/detection"
BATCH_SIZE = 64
EPOCHS = 3
MAX_LEMGTH = 128

In [5]:
def seed_everything(seed:int=42):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHEED"] = str(seed)
  tf.random.set_seed(seed)

seed_everything(SEED)

In [7]:
df_train = pd.read_csv("train.csv")
df_train["URL"] = df_train["URL"].str.replace(r"\[\.\]", ".", regex=True)

In [8]:
URLs = df_train["URL"].to_list()
labels = df_train["label"].to_list()

In [37]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
MAX_LENGTH = 128

def tokenze(text):
    tokens = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH,
        return_tensors='np'   # numpy로 반환
    )
    return tokens['input_ids'][0], tokens['attention_mask'][0]



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
input_ids = []
attention_masks = []
targets = []

max_samples = 10000  # 처리할 데이터 수 제한

for i, (url, label) in enumerate(tqdm(zip(URLs, labels), total=len(URLs))):
    if i >= max_samples:
        break
    try:
        _input_ids, _attention_mask = tokenize(url)
        input_ids.append(_input_ids)
        attention_masks.append(_attention_mask)
        targets.append(label)
    except Exception as e:
        print(f"Error at {url}: {e}")


  0%|          | 22565/6995056 [00:05<26:47, 4336.23it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-42-85d9f465fadb>", line 8, in <cell line: 0>
    _input_ids, _attention_mask = tokenize(url)
                                  ^^^^^^^^^^^^^
  File "<ipython-input-24-d9bffc03b702>", line 8, in tokenize
    tokenized = tokenizer(
                ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py", line 2867, in __call__
    encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py", line 2977, in _call_one
    return self.encode_plus(
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.p

In [42]:
imput_ids = np.array(input_ids)
attention_masks = np.array(attention_masks)
targets = np.array(targets)

In [43]:
def build_model():
  input_ids = tf.keras.layers.Input(shape(MAX_LEMGTH), dtype=tf.int32, name="input_ids")
  attention_mask = tf.keras.layers.Input(shape(MAX_LEMGTH), dtype=tf.int32, name="attention_mask")

  backbone = TFAutoModel.from_pretrained(MODEL_NAME)
  backbone.trainable = True

  x = backbone(
      input_ids=input_ids,
      attention_mask=attention_mask,
      straning=True
  ).last_hidden_state[:, 0, :]

  outputs = tf.keras.layers.Donse(1, activation="sigmoid", dtype=tf.float32, name="outputs")(x)
  model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
  return model

In [44]:
from transformers import TFAutoModel
from IPython.display import clear_output


class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:
            clear_output(wait=True)

callbacks = [
    DisplayCallback(),  # ← added comma
    tf.keras.callbacks.ModelCheckpoint(
        "./ckpts/v1",
        monitor="loss",
        mode="min",
        save_best_only=True
    )
]

# --- build_model with correct Input syntax ---
def build_model():
    # note: shape=(MAX_LENGTH,) not shape(MAX_LENGTH)
    input_ids      = tf.keras.layers.Input(shape=(MAX_LENGTH,),      dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(MAX_LENGTH,),      dtype=tf.int32, name="attention_mask")

    # load pretrained backbone
    backbone = TFAutoModel.from_pretrained(MODEL_NAME)
    # get the [CLS] token embedding
    sequence_output = backbone(
        input_ids,
        attention_mask=attention_mask
    )[0]  # outputs[0] is last_hidden_state: (batch, seq_len, hidden)
    cls_token = sequence_output[:, 0, :]  # (batch, hidden)

    # a simple binary classification head
    output = tf.keras.layers.Dense(1, activation="sigmoid")(cls_token)

    return tf.keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=output
    )

# --- compile & train ---
loss      = tf.keras.losses.BinaryFocalCrossentropy(from_logits=False,    # sigmoid already applied
                                                     # apply_class_balancing is deprecated, use class_weight if needed
                                                    )
optimizer = tf.keras.optimizers.AdamW(learning_rate=5e-5)
auc       = tf.keras.metrics.AUC()

model = build_model()
model.compile(loss=loss, optimizer=optimizer, metrics=[auc])

# assuming input_ids, attention_masks, targets are lists or numpy arrays
X = [input_ids, attention_masks]
y = targets

model.fit(
    X,
    y,
    epochs=EPOCHS,
    callbacks=callbacks,
    batch_size=BATCH_SIZE
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


ValueError: Data cardinality is ambiguous:
  x sizes: 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 1